# First 100 cases and policy enactment dates

- In this notebook we used "https://datahub.io/core/covid-19" to find the number of confirmed COVID-19 cases and deaths in the USA by state.
- We used this site "https://github.com/COVID19StatePolicy/SocialDistancing" to find policy data per state

### 1. Importing data for confirmed cases and deaths

In [1]:
import pandas as pd
import numpy as np

In [2]:
df_confirmed = pd.read_csv("raw_data/us_confirmed.csv")

df_confirmed.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Lat,Combined_Key,Date,Case,Long,Country/Region,Province/State
0,16,AS,ASM,16,60.0,NaN,-14.271,"American Samoa, US",2020-01-22,0,-170.132,US,American Samoa
1,16,AS,ASM,16,60.0,NaN,-14.271,"American Samoa, US",2020-01-23,0,-170.132,US,American Samoa
2,16,AS,ASM,16,60.0,NaN,-14.271,"American Samoa, US",2020-01-24,0,-170.132,US,American Samoa
3,16,AS,ASM,16,60.0,NaN,-14.271,"American Samoa, US",2020-01-25,0,-170.132,US,American Samoa
4,16,AS,ASM,16,60.0,NaN,-14.271,"American Samoa, US",2020-01-26,0,-170.132,US,American Samoa


In [3]:
df_deaths = pd.read_csv("raw_data/us_deaths.csv")

df_deaths.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Lat,Combined_Key,Population,Date,Case,Long,Country/Region,Province/State
0,16,AS,ASM,16,60.0,NaN,-14.271,"American Samoa, US",55641,2020-01-22,0,-170.132,US,American Samoa
1,16,AS,ASM,16,60.0,NaN,-14.271,"American Samoa, US",55641,2020-01-23,0,-170.132,US,American Samoa
2,16,AS,ASM,16,60.0,NaN,-14.271,"American Samoa, US",55641,2020-01-24,0,-170.132,US,American Samoa
3,16,AS,ASM,16,60.0,NaN,-14.271,"American Samoa, US",55641,2020-01-25,0,-170.132,US,American Samoa
4,16,AS,ASM,16,60.0,NaN,-14.271,"American Samoa, US",55641,2020-01-26,0,-170.132,US,American Samoa


### 2. Subset data to find date of first 100 cases of infections and deaths

In [4]:
df_first_hundred_infections = df_confirmed.loc[df_confirmed["Case"] >= 100].drop_duplicates(subset=["Province/State"], keep="first")
df_first_hundred_infections = df_first_hundred_infections[["Date","Case", "Province/State"]].reset_index(drop=True)
df_first_hundred_infections.columns = ["Date_infections","Infections", "State"]

df_first_hundred_infections.head()

,Date_infections,Infections,State
0,2020-04-05,112,Guam
1,2020-03-28,100,Puerto Rico
2,2020-07-04,111,Virgin Islands
3,2020-05-14,104,Alabama
4,2020-04-08,103,Alaska


In [5]:
df_first_hundred_deaths = df_deaths.loc[df_deaths["Case"] >= 100].drop_duplicates(subset=["Province/State"], keep="first")
df_first_hundred_deaths = df_first_hundred_deaths[["Date","Case", "Province/State"]].reset_index(drop=True)
df_first_hundred_deaths.columns = ["Date_deaths","Deaths", "State"]

df_first_hundred_deaths.head()

,Date_deaths,Deaths,State
0,2020-05-07,102,Puerto Rico
1,2020-05-28,100,Alabama
2,2020-04-23,115,Arizona
3,2020-06-05,101,California
4,2020-05-16,102,Colorado


In [6]:
# Merging infections and deaths data sets

df_infections_deaths = df_first_hundred_infections.merge(df_first_hundred_deaths, how="outer")

df_infections_deaths.head()

,Date_infections,Infections,State,Date_deaths,Deaths
0,2020-04-05,112,Guam,NaN,NaN
1,2020-03-28,100,Puerto Rico,2020-05-07,102.0
2,2020-07-04,111,Virgin Islands,NaN,NaN
3,2020-05-14,104,Alabama,2020-05-28,100.0
4,2020-04-08,103,Alaska,NaN,NaN


## 3. Importing policy data

- subset dataframe for "PublicMask" and "StayAtHome" policies
- find dates for when these policies were enacted


In [7]:
df_policies = pd.read_csv("raw_data/USstatesCov19distancingpolicy.csv", delimiter=";")
df_policies.head()

,location_id,StateFIPS,StatePostal,StateName,StatePolicy,Mandate,StateWide,DateIssued,DateEnacted,DateExpiry,DateEased,DateEnded,DateReexpanded1,PolicyCodingNotes,PolicySource,LastUpdated,LastUpdatedNotes
0,523,1,AL,Alabama,EmergDec,1,1,20200313,20200313,NaN,NaN,NaN,NaN,Public Health Emergency,https://governor.alabama.gov/newsroom/2020/03/...,20200324,NaN
1,523,1,AL,Alabama,SchoolClose,1,1,20200314,20200318,20200731.0,20200601.0,NaN,NaN,in effect 20200318. 20200319 issuance also clo...,http://alabamapublichealth.gov/legal/assets/42...,20200701,Updated DateExpiry to 20200731
2,523,1,AL,Alabama,GathRestrict25,1,1,20200319,20200319,NaN,NaN,NaN,NaN,"In effect as of 5 pm 20200319. On 20200320, am...",https://governor.alabama.gov/assets/2020/03/Al...,20200420,Updated DateEnacted to 20200319 per state heal...
3,523,1,AL,Alabama,GathRestrictAny,1,1,20200319,20200319,20200731.0,20200511.0,NaN,NaN,"In effect as of 5 pm 20200319. On 20200320, am...",https://governor.alabama.gov/assets/2020/03/Al...,20200701,Updated DateExpiry to 20200731
4,523,1,AL,Alabama,OtherBusinessClose,1,1,20200319,20200319,20200731.0,20200430.0,NaN,NaN,All beaches close effective at 5 pm 20200319,https://governor.alabama.gov/assets/2020/03/Al...,20200701,Updated DateExpiry to 20200731


In [8]:
# Subsetting for PublicMask and StayAtHome

df_policies = df_policies[df_policies["StatePolicy"].isin(["PublicMask", "StayAtHome"])]
df_policies.head()

,location_id,StateFIPS,StatePostal,StateName,StatePolicy,Mandate,StateWide,DateIssued,DateEnacted,DateExpiry,DateEased,DateEnded,DateReexpanded1,PolicyCodingNotes,PolicySource,LastUpdated,LastUpdatedNotes
9,523,1,AL,Alabama,StayAtHome,1,1,20200403,20200404,20200430.0,NaN,20200430.0,NaN,Goes into effect at 5 pm on 20200403. The 2020...,http://www.alabamapublichealth.gov/legal/asset...,20200501,Added DateEnded of 20200430
10,523,1,AL,Alabama,PublicMask,0,1,20200428,20200430,20200731.0,NaN,NaN,NaN,Recommendation from the Gov in the Safer At Ho...,http://www.alabamapublichealth.gov/legal/asset...,20200701,Updated DateExpiry to 20200731
11,523,1,AL,Alabama,StayAtHome,0,1,20200428,20200430,20200731.0,NaN,NaN,NaN,"Continues recommendation for all individuals, ...",https://governor.alabama.gov/assets/2020/04/Sa...,20200701,Updated DateExpiry to 20200731
16,524,2,AK,Alaska,StayAtHome,1,0,20200320,20200322,20200401.0,NaN,NaN,NaN,Only applicable to Anchorage. Enacted at 10 pm...,https://www.muni.org/departments/mayor/pressre...,20200623,"Updated DateExpiry, old date was 20200331, but..."
22,524,2,AK,Alaska,StayAtHome,1,1,20200327,20200328,NaN,NaN,20200424.0,NaN,In effect as of 5 pm on 20200328. The 20200424...,https://content.govdelivery.com/accounts/AKDHS...,20200426,Removed DateExpiry as 20200409 update removed ...


In [9]:
# Separating df into PublicMask and StayAtHome

df_public_mask = df_policies.loc[df_policies["StatePolicy"] == "PublicMask"].reset_index(drop=True)

df_public_mask.head()

,location_id,StateFIPS,StatePostal,StateName,StatePolicy,Mandate,StateWide,DateIssued,DateEnacted,DateExpiry,DateEased,DateEnded,DateReexpanded1,PolicyCodingNotes,PolicySource,LastUpdated,LastUpdatedNotes
0,523,1,AL,Alabama,PublicMask,0,1,20200428,20200430,20200731.0,NaN,NaN,NaN,Recommendation from the Gov in the Safer At Ho...,http://www.alabamapublichealth.gov/legal/asset...,20200701,Updated DateExpiry to 20200731
1,524,2,AK,Alaska,PublicMask,0,1,20200403,20200403,NaN,NaN,NaN,NaN,Highly recommended for all Alaskans: wear a cl...,http://dhss.alaska.gov/dph/Epi/id/SiteAssets/P...,20200426,NaN
2,526,5,AR,Arkansas,PublicMask,0,1,20200423,20200423,NaN,NaN,NaN,NaN,"Everyone should wear a face covering, or mask,...",https://www.healthy.arkansas.gov/images/upload...,20200623,Updated PolicyNotes re extension
3,527,6,CA,California,PublicMask,0,1,20200401,20200401,NaN,NaN,NaN,NaN,Provides guidance around using face coverings ...,https://www.cdph.ca.gov/Programs/CID/DCDC/Page...,20200525,NaN
4,527,6,CA,California,PublicMask,1,1,20200618,20200618,NaN,NaN,NaN,NaN,(quote) People in California must wear face co...,https://www.cdph.ca.gov/Programs/CID/DCDC/CDPH...,20200621,NaN


In [10]:
df_stay_home = df_policies.loc[df_policies["StatePolicy"] == "StayAtHome"]
df_stay_home.head()

,location_id,StateFIPS,StatePostal,StateName,StatePolicy,Mandate,StateWide,DateIssued,DateEnacted,DateExpiry,DateEased,DateEnded,DateReexpanded1,PolicyCodingNotes,PolicySource,LastUpdated,LastUpdatedNotes
9,523,1,AL,Alabama,StayAtHome,1,1,20200403,20200404,20200430.0,NaN,20200430.0,NaN,Goes into effect at 5 pm on 20200403. The 2020...,http://www.alabamapublichealth.gov/legal/asset...,20200501,Added DateEnded of 20200430
11,523,1,AL,Alabama,StayAtHome,0,1,20200428,20200430,20200731.0,NaN,NaN,NaN,"Continues recommendation for all individuals, ...",https://governor.alabama.gov/assets/2020/04/Sa...,20200701,Updated DateExpiry to 20200731
16,524,2,AK,Alaska,StayAtHome,1,0,20200320,20200322,20200401.0,NaN,NaN,NaN,Only applicable to Anchorage. Enacted at 10 pm...,https://www.muni.org/departments/mayor/pressre...,20200623,"Updated DateExpiry, old date was 20200331, but..."
22,524,2,AK,Alaska,StayAtHome,1,1,20200327,20200328,NaN,NaN,20200424.0,NaN,In effect as of 5 pm on 20200328. The 20200424...,https://content.govdelivery.com/accounts/AKDHS...,20200426,Removed DateExpiry as 20200409 update removed ...
37,525,4,AZ,Arizona,StayAtHome,1,1,20200330,20200331,20200516.0,NaN,20200516.0,NaN,Extended stay-at-home to 20200516 https://azgo...,https://azgovernor.gov/executive-orders/eo_202...,20200512,Updated DateEnded to 20200516


In [11]:
# subset for dates when Public Mask policy was enacted and eased

df_public_mask = df_public_mask[["StateName","Mandate", "StateWide", "DateEnacted", "DateEased"]]

df_public_mask.head()

,StateName,Mandate,StateWide,DateEnacted,DateEased
0,Alabama,0,1,20200430,NaN
1,Alaska,0,1,20200403,NaN
2,Arkansas,0,1,20200423,NaN
3,California,0,1,20200401,NaN
4,California,1,1,20200618,NaN


In [12]:
# subset for dates when Stay at Home policy was enacted and eased

df_stay_home = df_stay_home[["StateName","Mandate", "StateWide", "DateEnacted", "DateEased"]]

df_stay_home.head()

,StateName,Mandate,StateWide,DateEnacted,DateEased
9,Alabama,1,1,20200404,NaN
11,Alabama,0,1,20200430,NaN
16,Alaska,1,0,20200322,NaN
22,Alaska,1,1,20200328,NaN
37,Arizona,1,1,20200331,NaN


In [13]:
# Changing "DateEnacted" to "PublicMask_date"
# Changing "DateEased" to "PublicMask_end" 
# Converting to datetime

df_public_mask.columns = ["State","Mandate_mask", "StateWide_mask", "PublicMask_date", "PublicMask_end"]

df_public_mask["PublicMask_date"] = pd.to_datetime(df_public_mask["PublicMask_date"], format="%Y%m%d")
df_public_mask["PublicMask_end"] = pd.to_datetime(df_public_mask["PublicMask_end"], format="%Y%m%d")

df_public_mask = df_public_mask.drop_duplicates(subset=["State"], keep="first")

df_public_mask.head()

,State,Mandate_mask,StateWide_mask,PublicMask_date,PublicMask_end
0,Alabama,0,1,2020-04-30,NaT
1,Alaska,0,1,2020-04-03,NaT
2,Arkansas,0,1,2020-04-23,NaT
3,California,0,1,2020-04-01,NaT
5,Colorado,0,1,2020-04-27,NaT


In [14]:
# Changing "DateEnacted" to "StayAtHome_date" and converting to datetime

df_stay_home.columns = ["State","Mandate_home", "StateWide_home", "StayAtHome_date", "StayAtHome_end"]
df_stay_home["StayAtHome_date"] = pd.to_datetime(df_stay_home["StayAtHome_date"], format="%Y%m%d")
df_stay_home["StayAtHome_end"] = pd.to_datetime(df_stay_home["StayAtHome_end"], format="%Y%m%d")

df_stay_home = df_stay_home.drop_duplicates(subset=["State"], keep="first").reset_index(drop=True)

df_stay_home.head()

,State,Mandate_home,StateWide_home,StayAtHome_date,StayAtHome_end
0,Alabama,1,1,2020-04-04,NaT
1,Alaska,1,0,2020-03-22,NaT
2,Arizona,1,1,2020-03-31,NaT
3,California,1,1,2020-03-19,NaT
4,Colorado,1,1,2020-03-26,2020-04-27


## 4. Merge policy date dataframes

- merge dataframes
- subtract policy dates from dates of first 100 cases per state

In [15]:
df_mask_and_stay_home = df_public_mask.merge(df_stay_home, how="outer")
df_mask_and_stay_home.head(10)

,State,Mandate_mask,StateWide_mask,PublicMask_date,PublicMask_end,Mandate_home,StateWide_home,StayAtHome_date,StayAtHome_end
0,Alabama,0.0,1.0,2020-04-30,NaT,1.0,1.0,2020-04-04,NaT
1,Alaska,0.0,1.0,2020-04-03,NaT,1.0,0.0,2020-03-22,NaT
2,Arkansas,0.0,1.0,2020-04-23,NaT,NaN,NaN,NaT,NaT
3,California,0.0,1.0,2020-04-01,NaT,1.0,1.0,2020-03-19,NaT
4,Colorado,0.0,1.0,2020-04-27,NaT,1.0,1.0,2020-03-26,2020-04-27
5,Connecticut,1.0,1.0,2020-04-20,NaT,0.0,1.0,2020-03-23,NaT
6,Delaware,1.0,1.0,2020-04-28,NaT,1.0,1.0,2020-03-24,2020-06-01
7,District of Columbia,1.0,1.0,2020-04-17,2020-06-22,1.0,1.0,2020-04-01,NaT
8,Florida,0.0,1.0,2020-06-20,NaT,1.0,1.0,2020-04-03,2020-05-04
9,Georgia,0.0,1.0,2020-05-01,NaT,1.0,0.0,2020-03-24,NaT


In [16]:
df_final = df_infections_deaths.merge(df_mask_and_stay_home, on="State", how="left")

df_final.head()

,Date_infections,Infections,State,Date_deaths,Deaths,Mandate_mask,StateWide_mask,PublicMask_date,PublicMask_end,Mandate_home,StateWide_home,StayAtHome_date,StayAtHome_end
0,2020-04-05,112,Guam,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaT,NaT
1,2020-03-28,100,Puerto Rico,2020-05-07,102.0,NaN,NaN,NaT,NaT,NaN,NaN,NaT,NaT
2,2020-07-04,111,Virgin Islands,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaT,NaT
3,2020-05-14,104,Alabama,2020-05-28,100.0,0.0,1.0,2020-04-30,NaT,1.0,1.0,2020-04-04,NaT
4,2020-04-08,103,Alaska,NaN,NaN,0.0,1.0,2020-04-03,NaT,1.0,0.0,2020-03-22,NaT


In [17]:
df_final["Date_infections"] = pd.to_datetime(df_final["Date_infections"])

In [18]:
df_final["Date_deaths"] = pd.to_datetime(df_final["Date_deaths"])

In [19]:
# Number of days difference from first 100 infections to Stay Home policy enactment

df_final = df_final.assign(StayAtHome_Delta = pd.to_timedelta(np.where(df_final["StayAtHome_date"].notna(), 
                                                        df_final["StayAtHome_date"] - df_final["Date_infections"], 
                                                        pd.NA)))

In [20]:
# Number of days difference from first 100 infections to Public Mask policy enactment

df_final = df_final.assign(PublicMask_Delta = pd.to_timedelta(np.where(df_final["PublicMask_date"].notna(), 
                         df_final["PublicMask_date"] - df_final["Date_infections"], 
                         pd.NA)))

In [21]:
# Number of days difference from first 100 deaths to Stay Home policy enactment

df_final = df_final.assign(StayAtHome_deaths_Delta = pd.to_timedelta(np.where(df_final["StayAtHome_date"].notna(), 
                                                        df_final["StayAtHome_date"] - df_final["Date_deaths"], 
                                                        pd.NA)))

In [22]:
# Number of days difference from first 100 infections to Public Mask policy enactment

df_final = df_final.assign(PublicMask_deaths_Delta = pd.to_timedelta(np.where(df_final["PublicMask_date"].notna(), 
                         df_final["PublicMask_date"] - df_final["Date_deaths"], 
                         pd.NA)))

In [23]:
# Length of Public Mask Policy

df_final = df_final.assign(PublicMask_policy_days = pd.to_timedelta(np.where(df_final["PublicMask_date"].notna(), 
                         df_final["PublicMask_end"] - df_final["PublicMask_date"], 
                         pd.NA)))

In [24]:
# Length of Stay at Home Policy

df_final = df_final.assign(StayAtHome_policy_days = pd.to_timedelta(np.where(df_final["StayAtHome_date"].notna(), 
                         df_final["StayAtHome_end"] - df_final["StayAtHome_date"], 
                         pd.NA)))

In [25]:
df_final.head(20)

,Date_infections,Infections,State,Date_deaths,Deaths,Mandate_mask,StateWide_mask,PublicMask_date,PublicMask_end,Mandate_home,StateWide_home,StayAtHome_date,StayAtHome_end,StayAtHome_Delta,PublicMask_Delta,StayAtHome_deaths_Delta,PublicMask_deaths_Delta,PublicMask_policy_days,StayAtHome_policy_days
0,2020-04-05,112,Guam,NaT,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
1,2020-03-28,100,Puerto Rico,2020-05-07,102.0,NaN,NaN,NaT,NaT,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
2,2020-07-04,111,Virgin Islands,NaT,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
3,2020-05-14,104,Alabama,2020-05-28,100.0,0.0,1.0,2020-04-30,NaT,1.0,1.0,2020-04-04,NaT,-40 days,-14 days,-54 days,-28 days,NaT,NaT
4,2020-04-08,103,Alaska,NaT,NaN,0.0,1.0,2020-04-03,NaT,1.0,0.0,2020-03-22,NaT,-17 days,-5 days,NaT,NaT,NaT,NaT
5,2020-04-15,110,Arizona,2020-04-23,115.0,NaN,NaN,NaT,NaT,1.0,1.0,2020-03-31,NaT,-15 days,NaT,-23 days,NaT,NaT,NaT
6,2020-04-29,101,Arkansas,NaT,NaN,0.0,1.0,2020-04-23,NaT,NaN,NaN,NaT,NaT,NaT,-6 days,NaT,NaT,NaT,NaT
7,2020-03-23,118,California,2020-06-05,101.0,0.0,1.0,2020-04-01,NaT,1.0,1.0,2020-03-19,NaT,-4 days,9 days,-78 days,-65 days,NaT,NaT
8,2020-03-29,110,Colorado,2020-05-16,102.0,0.0,1.0,2020-04-27,NaT,1.0,1.0,2020-03-26,2020-04-27,-3 days,29 days,-51 days,-19 days,NaT,32 days
9,2020-03-19,102,Connecticut,2020-04-06,101.0,1.0,1.0,2020-04-20,NaT,0.0,1.0,2020-03-23,NaT,4 days,32 days,-14 days,14 days,NaT,NaT


In [26]:
df_final.to_csv("data/mask_and_lockdown_policy_data.csv")